# Quadrotor Landing Simulation on Oscillating Platform

This notebook implements a **closed-loop simulation** of a quadrotor attempting to land
on an oscillating platform. It integrates the system dynamics, generates reference trajectories,
and evaluates control strategies such as **MPC** and **LQR**.

The project models:
- Full nonlinear quadrotor dynamics with **ground effect**, **wind disturbances**, and **aerodynamic drag**.
- A moving landing platform with configurable oscillation patterns.
- Controllers to track desired trajectories and achieve safe landing.

Use this notebook to:
1. Configure the simulation environment.
2. Define model and trajectory parameters.
3. Run the closed-loop simulation.
4. Visualize and analyze the quadrotor's landing performance.

## Environment Configuration

Before running any simulation or control routines, you need to configure the environment.  
This step loads all required packages, sets up paths, and initializes the project settings.  

> **Note:** Run this cell **only once** at the start of your session to avoid reloading
> dependencies multiple times.

In [ ]:
# Configures the Environment (run it once)
include("../src/configure.jl")

## Load Modules and Configure Model Parameters

In this section, we:

1. **Include the core source files**, which define the main components of the simulation:
   - `Model_nd_Dynamics.jl`: Quadrotor physical model and dynamics.
   - `Simulation.jl`: Simulation configuration and timing.
   - `MPCTunningParams.jl`: Parameters for tuning the MPC controller.
   - `Trajectory.jl`: Trajectory generation for the platform and quadrotor.
   - `MPC.jl`: MPC controller implementation.
   - `Close_loop.jl`: Closed-loop simulation logic.
   - `LQR.jl`: LQR controller implementation.

In [ ]:
include("../src/Model_nd_Dynamics.jl")
include("../src/Simulation.jl")
include("../src/MPCTunningParams.jl")
include("../src/Trajectory.jl")
include("../src/MPC.jl")
include("../src/Close_loop.jl")
include("../src/LQR.jl")

2. **Initialize the quadrotor model** with its physical parameters, including:
   - Mass, gravity, geometry, inertia matrix.
   - Propeller properties and thrust limits.
   - Aerodynamic drag matrix.

3. **Configure the simulation**, defining:
   - Time step for the plant (`h_universe`).
   - Time step for the controller (`h_controller`).
   - Total simulation time (`Tfinal`).

4. **Set up the trajectory parameters**, including:
   - Initial height of the quadrotor.
   - Stationary height for landing.
   - Oscillation amplitude and frequency of the landing platform.

5. **Define controller tuning**, such as:
   - Prediction horizon (`Nh`) and control horizon (`Nc`).
   - Cost weights (`Q` and `R`) for the MPC controller.

6. **Specify initial conditions**, including:
   - Initial state vector `x0` with position, orientation, velocities, etc.
   - Wind vector and velocity for environmental disturbances.

In [ ]:
model = Model(0.0, 0.0, 0.0, I(3), 0.0, 0.0, 0.0, zeros(4), zeros(4),0.0,0.0, 0, 0, I(3))
model.g = 9.81 # [m/s^2]
model.m = 0.028 # [kg]
model.ℓ = 0.0460 # [meters]
model.J = 1e-6 * [
    16.571710   0.830806   0.718277
     0.830806  16.655602   1.800197
     0.718277   1.800197  29.261652
] # [kg . m^2]
model.R = 0.0235 # [meters]
model.prop_min_h = 0.1 # [meters]
model.ρ = 3.6 # dimensionless
model.kt = 1.28e-8
model.km = model.kt * 0.00596
model.Nx = 13
model.Nu = 4

#Thrust limits
model.umin = 0 * model.g * ones(4)
model.umax = 0.58 * model.m * model.g * ones(4)

model.K_aero = 1e-7 * [ 9.1785   0.0000   0.0000;
                         0.0000   9.1785   0.0000;
                         0.0000   0.0000   10.311 ]

## Simulation Configs
h_universe = 0.0001 # Time step (10 kHz)
h_controller = 0.002 # Time step (500 Hz)
Tfinal = 10.0 # Simulate from 0 - 10 seconds

simulation = Simulation(h_universe, h_controller, Tfinal)

landing_start_height = 3.0
stationary_height = model.prop_min_h + 0.06 # 6 centimeters above the ground
plat_amplitude = 0.1
plat_freq = 0.3

traj_params = Trajectory(plat_amplitude, plat_freq, landing_start_height, stationary_height)

state_space = LinearStateSpace(model, simulation, (model.Nx - 1), model.Nu, stationary_height)

Nh = 20
Nc = 19 # Goes from 0 to (Nh -1)

# Cost weights
Q = Array(100.0*I(model.Nx - 1));
R = Array(.1*I(model.Nu));

tunning_params = MPCTunningParameters(Nh, Nc, Q, R)

mpc_mats =  MPCMatrices(model, state_space, tunning_params)

# Initial State!
theta = 45 * pi / 180  # from degrees to rad
q0 = [cos(theta/2), sin(theta/2), 0.0, 0.0]
x0 = [1.0; 2.0; traj_params.init_heigth; q0 ; 0.0; 0.0; 0.0; 0.0; 0.0; 0.0]

wind_vec = [0.0, 1.0, 0.0]
wind_velocity = 0.0; # m/s

## Run Closed-Loop Simulations (MPC vs LQR)

This section runs two full-duration simulations of the quadrotor–platform system:

- **`out` (MPC):** Uses an anonymous controller that calls `mpc_controller(...)` each controller step,
  passing the current time/state and the reference `gen_ref(...)` computed at `h_controller`.
- **`out_lqr` (LQR):** Same setup, but using `lqr_controller(...)`.

Both runs:
- Use identical initial conditions, model, simulation, and wind settings.
- Execute `closed_loop(...)`, which logs states, inputs, references, platform states,
  landing markers, and timing into a `SimResults` struct.
- Allow fair performance comparison between MPC and LQR under the same conditions.

In [ ]:
out = closed_loop(
    x0,
    state_space,
    tunning_params, 
    traj_params,
    simulation,
    model,

    (t, x) -> mpc_controller(
        model,
        simulation,
        tunning_params,
        traj_params,
        state_space,
        mpc_mats,
        t,
        x, 
        gen_ref(model, traj_params, tunning_params, t, simulation.h_controller) # verify if this works as expected!!
    ),
    wind_vec,
    wind_velocity
);

In [ ]:
out_lqr = closed_loop(
    x0,
    state_space,
    tunning_params, 
    traj_params,
    simulation,
    model,

    (t, x) -> lqr_controller(
        model,
        simulation,
        tunning_params,
        traj_params,
        state_space,
        t,
        x, 
        gen_ref(model, traj_params, tunning_params, t, simulation.h_controller) # verify if this works as expected!!
    ),
    wind_vec,
    wind_velocity
);

## Results Visualization

This section generates plots to **analyze and compare** the performance of the two controllers
(MPC and LQR) over the full simulation.

- **Position states (`x`, `y`, `z`)** and **velocities (`dx`, `dy`, `dz`)** are plotted to
  evaluate tracking performance and landing behavior.
- **Attitude error** is shown to measure the quadrotor's orientation accuracy.
- **Control inputs (`u₁` to `u₄`)** are visualized to verify that motor thrusts remain within
  the defined limits.

In [ ]:
include("../src/Report.jl")

thist = Array(range(0, simulation.h_universe * (simulation.Nt_universe - 1), step = simulation.h_universe));

In [ ]:
plot_state(1, "x", "m", out, out_lqr, thist)

In [ ]:
plot_state(2, "y", "m", out, out_lqr, thist)

In [ ]:
plot_state(3, "z", "m", out, out_lqr, thist)

In [ ]:
plot_state(8, "dx", "m/s", out, out_lqr, thist)

In [ ]:
plot_state(9, "dy", "m/s", out, out_lqr, thist)

In [ ]:
plot_state(10, "dz", "m/s", out, out_lqr, thist)

In [ ]:
plot_attitude_error(out, out_lqr, thist)

In [ ]:
plot_u(1, out, out_lqr, thist, model)

In [ ]:
plot_u(2, out, out_lqr, thist, model)

In [ ]:
plot_u(3, out, out_lqr, thist, model)

In [ ]:
plot_u(4, out, out_lqr, thist, model)

## 3D Trajectory Playback (MeshCat)

This cell sets up an interactive **3D visualization** of the quadrotor flight path using
`MeshCat` and `TrajOptPlots`.

- Loads a quadrotor mesh and opens a WebGL viewer in the notebook.
- Samples the simulated states every 10 steps (`1:10:end`) to keep playback smooth.
- Plays the motion for the full simulation duration (`thist[end]`).

> **Note:** Only the **quadrotor** is rendered here—the oscillating **platform is not shown**.
> Use these visuals as an alternative way to explore the flown trajectory.

In [ ]:
#Set up visualization
using TrajOptPlots
using MeshCat
using StaticArrays
using RobotZoo:Quadrotor

vis = Visualizer()
render(vis)

In [ ]:
quad_model = Quadrotor()
TrajOptPlots.set_mesh!(vis, quad_model)

In [ ]:
X1 = [SVector{13}(x) for x in eachcol(out.x_quad[:,1:10:end])];

visualize!(vis, quad_model, thist[end], X1)